Import libraries:

In [1]:
from __future__ import unicode_literals
from nltk.tokenize import word_tokenize
import wikipediaapi
import string

from rouge import Rouge
from rouge_score import rouge_scorer

import matplotlib.pyplot as plt
from  more_itertools import unique_everseen

import pandas as pd
import numpy as np
import fuzzy_pandas as fpd
from collections import Counter
from webdriver_manager.chrome import ChromeDriverManager

from difflib import SequenceMatcher
from rouge_score import rouge_scorer

import os, time, random
import docx, html, re, nltk
import collections

from tqdm import tqdm
from docx import Document
from docx.shared import Cm
from docx.shared import Pt

import concurrent.futures
from docx.enum.dml import MSO_THEME_COLOR_INDEX
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Pt

from reportlab.lib.styles import ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.enums import TA_JUSTIFY, TA_CENTER
from reportlab.platypus import Table, TableStyle
from reportlab.lib.pagesizes import letter
from reportlab.lib.units import inch
from reportlab.platypus import Image
from reportlab import platypus

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

from tika import parser
from gensim.summarization.summarizer import summarize

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
from requests import get

nltk.download('punkt')
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from urllib.request import urlopen
from bs4 import BeautifulSoup
from bs4.element import Comment

import networkx as nx
from nltk.tokenize import sent_tokenize

import spacy
import neuralcoref
from spacy.symbols import nsubj, nsubjpass, VERB

from selenium.webdriver.common.keys import Keys
import chromedriver_binary

from dateutil.relativedelta import relativedelta
from datetime import datetime as dt

from selenium import webdriver
from bs4 import BeautifulSoup
from bs4.element import Tag
from random import randint
import winsound

nlp = spacy.load('en_core_web_lg')
nlp.max_length = 50000000

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\skamenshchikov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Import CDFs:

In [33]:
##### HTML parsing #####
def parse_page(url, tag, cls=''): 
    soup = BeautifulSoup(get(url).text, 'html.parser')
    paragraphs = soup.find('body', cls='')
    
    title = BeautifulSoup(get(url).text, 'html.parser').title.getText()
    
    parser = HtmlParser.from_url(url, Tokenizer("English"))
    summarizer = Summarizer(Stemmer("English"))
    summarizer.stop_words = get_stop_words("English")

    sentences = []
    for i in summarizer(parser.document, 1000000):
        sentences.append(str(i))
    
    txt = ' '.join(sentences)  
    
    return txt, title

def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)

def get_unique_text(document):
    unique_sentences = []
    for sentence in [sent.raw for sent in TextBlob(document).sentences]:
        if sentence not in unique_sentences:
            unique_sentences.append(sentence)
    return ' '.join(unique_sentences)

def get_text(url):
    page = urlopen(url)
    soup = BeautifulSoup(page)
    fetched_text = ' '.join(map(lambda p:p.text,soup.find_all('p')))
    return fetched_text
#####/HTML parsing #####

##### Text preprocessing #####
def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return u" ".join(t.strip() for t in visible_texts)

def text_normalize(txt):
    processed_text = re.sub('[^a-zA-Z]', ' ', txt)
    processed_text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",processed_text)
    processed_text=re.sub("(\\d|\\W)+"," ",processed_text)

    stop_words = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(processed_text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if not word in stop_words]
    tokens = [i for i in tokens if (tags(i) in ['NN', 'NNP', 'NNS', 'NNPS'])]

    return tokens

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def filter_triplet(final_text):
    final_text = get_unique_text(final_text)
    doc = nlp(final_text)
    valid_sents=[]

    for s in list(doc.sents):
        if syntax_full(s):
            valid_sents.append(s.text)
    final_text = ' '.join(valid_sents)
    return final_text

def coref_res(rawtext, coref_greedn = 0.5):

    neuralcoref.add_to_pipe(nlp, greedyness = coref_greedn, store_scores=False)
    doc = nlp(rawtext)

    resolved = list(tok.text_with_ws for tok in doc)

    for cluster in doc._.coref_clusters:
        for coref in cluster:
            if coref != cluster.main:
                if coref.text[0].isalpha() and coref.text[0].isupper():

                    main_words_list=word_tokenize(cluster.main.text)
                    main_words_list[0]=main_words_list[0].capitalize()
                    resolved[coref.start] = detokenizer(main_words_list) + doc[coref.end-1].whitespace_

                for i in range(coref.start+1, coref.end):
                    resolved[i] = ""
            else:
                resolved[coref.start] = cluster.main.text + doc[coref.end-1].whitespace_
                for i in range(coref.start+1, coref.end):
                    resolved[i] = ""

    text_resolved = ''.join(resolved)
    nlp.remove_pipe("neuralcoref")

    return text_resolved

def compress(spacy_sents,sents_whitelist):
    blacklist_tokens=[]
    n=1
    for sent in spacy_sents:
        if (n in sents_whitelist):
            for token in sent:
                if token.dep_ in ['appos','advmod']:
                    token_sub_tree=token.subtree
                    for t in token_sub_tree:
                        blacklist_tokens.append(t.i)

        n=n+1
    return(blacklist_tokens)

def spacy_compress(rawtext):

    doc1 = nlp(rawtext)
    sents_whitelist = get_sents_ids_whitelist(doc1.sents)

    tokens_blacklist = compress(doc1.sents,sents_whitelist)
    sents_tokens = get_list_sents_tokens(doc1.sents,sents_whitelist,tokens_blacklist)
    compressed_text_sents = []

    for s in sents_tokens:
        text=detokenizer(s)
        compressed_text_sents.append(text)
    compressed_text_sents=sentence_grammar_fix(compressed_text_sents)
    text =' '.join(compressed_text_sents)

    return(text)
##### Text preprocessing #####

def readingTime(mytext):
    total_words = len(word_tokenize(mytext))
    estimatedTime = round(total_words/200.0,1)
    return estimatedTime

def grey_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    return "hsl(0, 0%%, %d%%)" % random.randint(60, 100)

def tags(x):
    return nltk.pos_tag(nltk.word_tokenize(x))[0][1]

def syntax_full(spacy_sentence):
    result=[]
    for token in spacy_sentence:
        if (token.dep == nsubj or token.dep == nsubjpass) and token.head.pos == VERB:
            result.append(token.head)
    if result:
        return True
    else:
        return False

def check_min_num_of_clauses(spacy_sentence, n):
    result=[]
    for token in spacy_sentence:
        if (token.dep_ in ['nsubj','nsubjpass','csubj','expl']) and (token.head.pos_ == 'VERB' or token.head.pos_ == 'AUX'):
            result.append(token.head.text)
    if len(result)>=n:
        return True

    else:
        return False

def get_sents_ids_whitelist(spacy_sents):
    whitelist=[]
    i=1
    sents_texts=[]
    for sent in spacy_sents:
        if (sent.text not in sents_texts) and check_min_num_of_clauses(sent,1):
            whitelist.append(i)
            sents_texts.append(sent.text)
        i=i+1
    return(whitelist)

def get_list_sents_tokens(spacy_sents,sents_whitelist,blacklist_tokens):
    sents_tokens=[]
    n=1
    for sent in spacy_sents:
        sent_tokens=[]
        if (n in sents_whitelist):
            for token in sent:
                if (token.i not in blacklist_tokens):
                    sent_tokens.append(token.text)
            sents_tokens.append(sent_tokens)
            sent_tokens=[]

        n=n+1
    return(sents_tokens)

def detokenizer(list_of_tokens):
    text_str="".join([" "+w if not w.startswith("'") and not w.startswith("’") and w!='' and w not in string.punctuation else w for w in list_of_tokens]).strip()
    return(text_str)

def sentence_grammar_fix(sentences):
    fixed=[]
    for sent in sentences:

        sent=sent.strip()
        sent=sent.replace('\n','')
        sent=sent.replace('()','')

        sent=re.sub('\s+',' ',sent)
        sent=sent+'.'
        sent=re.sub(r'([,.\-—:])+',r'\1',sent)

        if len(sent)>1:
            if sent[0] in ['.',',','-','—']:
                sent=sent[1:]
        sent=sent.strip()

        if len(sent)>1:
            if sent[0].isalpha():
                sent=sent[0].upper()+sent[1:]
        fixed.append(sent)

    return(fixed)

############# Parse Wiki ############# 
def parse_wiki(google_url):
    
    # load driver
    driver = webdriver.Chrome(ChromeDriverManager().install())
    #/load driver
    
    # get urls  
    driver.get(google_url)
    time.sleep(randint(1,5))

    soup = BeautifulSoup(driver.page_source,'lxml')
    result_div = soup.find_all('div', attrs={'class': 'g'})

    links = []
    titles = []
    errors = []

    descriptions = []
    for r in result_div:
        try:
            link = r.find('a', href=True)
            title = None
            title = r.find('h3')

            if isinstance(title,Tag):
                title = title.get_text()

            description = None
            description = r.find('span', attrs={'class': 'st'})

            if isinstance(description, Tag):
                description = description.get_text()

            if link != '' and title != '' and description != '':
                links.append(link['href'])
                titles.append(title)
                descriptions.append(description)

        except Exception as e:
            print(e)
            continue

    url_list = links[:(page_number)]
    url_list = [i for i in url_list if 'https://en.wikipedia.org' in i]
    
    title_list = []
    for i in url_list:
        try:
            if 'https://en.wikipedia.org' in i: 
                title_list.append(i.split('/')[4])  
        except:
            continue
    #/ get urls
        
    driver.stop_client()
    driver.close()
    
    return title_list     
############# Parse Wiki ############# 

############# Parse Arxiv #############
def parse_arxiv(query, page_number):
    arxivtext = ''
    
    urls = []
    titles = []

    closest_value = 100
    req = 'https://arxiv.org/search/?query='+query+'&size='+str(closest_value) + '&searchtype=all&source=header&start=0'
    htmlString = get(req)

    soup = BeautifulSoup(htmlString.content, 'html5lib')
    hrefs = soup.find_all('a', {'href': re.compile(r'arxiv.org/abs/')})

    titles = list(soup.find_all('p', {'class' : 'title is-5 mathjax'}))[:page_number]
    titles_r = [i.text.replace('\n','').replace('  ','') for i in titles]
    titles = ', '.join(titles_r)

    if (len(hrefs) > 0):
        for i in hrefs:
            urls.append(i['href'])

    txt = []
    for i in tqdm(urls[:page_number]):
        
        time.sleep(random.randint(1,8))

        soup = BeautifulSoup(get(str(i)).content, 'html5lib')
        abstract = soup.find('blockquote').text.replace('  ',' ')
        parsed_pdf = parser.from_file(str(i).replace('abs','pdf'))['content']  

        extended_abs = ' '.join(filter_text(parsed_pdf, abstract, threshold=0.5))    
        txt.append(extended_abs.replace('\n', ' ').replace('Abstract', '').replace('ABSTRACT', ''))

    arxivtext = re.sub('[^A-Za-z0-9.]+', ' ', '; '.join(txt))
    df = pd.DataFrame(list(zip(txt, urls, titles_r)), columns=['text','link', 'page'])

    return arxivtext, titles, df
#############/Parse Arxiv #############

def longest_common_substring(s1, s2):
  m = [[0] * (1 + len(s2)) for i in range(1 + len(s1))]
  longest, x_longest = 0, 0
  for x in range(1, 1 + len(s1)):
    for y in range(1, 1 + len(s2)):
      if s1[x - 1] == s2[y - 1]:
        m[x][y] = m[x - 1][y - 1] + 1
        if m[x][y] > longest:
          longest = m[x][y]
          x_longest = x
      else:
        m[x][y] = 0
  return s1[x_longest - longest: x_longest]

def longest_common_sentence(s1, s2):
    s1_words = s1.split(' ')
    s2_words = s2.split(' ')
    return ' '.join(longest_common_substring(s1_words, s2_words))

def css(a,b):
    if len(a.split()) > 0:
        score = len(longest_common_sentence(a,b).split())/len(a.split())
    else:    
        score = 0
    return score 
#/common string #

############# Parse Google #############
def parse_google(system, query, keys_number, page_number, tag, cls=''):   
    
    # load driver
    driver = webdriver.Chrome(ChromeDriverManager().install())
    #/load driver 

    # get urls
    google_url = system + query + "&num=" + str(page_number+1)
    google_url = google_url + '&hl=en&gl=en' + '&lr=lang_en&cr=countryGB'
    
    driver.get(google_url)
    time.sleep(randint(1,5))

    soup = BeautifulSoup(driver.page_source,'lxml')
    result_div = soup.find_all('div', attrs={'class': 'g'})

    links = []
    titles = []
    errors = []

    descriptions = []
    for r in result_div:
        try:
            link = r.find('a', href=True)
            title = None
            title = r.find('h3')

            if isinstance(title,Tag):
                title = title.get_text()

            description = None
            description = r.find('span', attrs={'class': 'st'})

            if isinstance(description, Tag):
                description = description.get_text()

            if link != '' and title != '' and description != '':
                links.append(link['href'])
                titles.append(title)
                descriptions.append(description)

        except Exception as e:
            print(e)
            continue

    url_list = links[:(page_number)]
    #/ get urls

    # load content 
    txt = []
    titles = []
    for j in tqdm(url_list):
        delta = random.randint(1,1)
        time.sleep(delta)
        
        try:  
            if str(j).endswith('.pdf'): 
                file_data = parser.from_file(str(j))           
                t = file_data['content']
                titles.append(t[:100])
            else:
                t = parse_page(j,tag,cls)[0].replace('\n','')
                titles.append(parse_page(j,tag,cls)[1])
            
            txt.append(t)
            
        except:
            print('Parsing error:',str(j))
            errors.append(str(j))
       
    googletext = re.sub('[^A-Za-z0-9.]+', ' ', '; '.join(txt))
    df = pd.DataFrame(list(zip(txt, url_list, titles)), columns=['text','link', 'page'])
    
    driver.stop_client()
    driver.close()
   
    return googletext, errors, df, titles
#############/Parse Google #############

############# Doc preparation ##########
def add_hyperlink(paragraph, text, url, flag):
    part = paragraph.part
    r_id = part.relate_to(url, docx.opc.constants.RELATIONSHIP_TYPE.HYPERLINK, is_external=True)

    # Create the w:hyperlink tag and add needed values
    hyperlink = docx.oxml.shared.OxmlElement('w:hyperlink')
    hyperlink.set(docx.oxml.shared.qn('r:id'), r_id, )

    # Create a w:r element and a new w:rPr element
    new_run = docx.oxml.shared.OxmlElement('w:r')
    rPr = docx.oxml.shared.OxmlElement('w:rPr')

    # Join all the xml elements together add add the required text to the w:r element
    new_run.append(rPr)
    new_run.text = text
    hyperlink.append(new_run)

    # Create a new Run object and add the hyperlink into it
    r = paragraph.add_run()
    r._r.append (hyperlink) 

    # A workaround for the lack of a hyperlink style (doesn't go purple after using the link)
    # Delete this if using a template that has the hyperlink style in it
    r.font.color.theme_color = MSO_THEME_COLOR_INDEX.HYPERLINK
    r.font.underline = flag

    return hyperlink

def save_doc(final_summary, summary, query, docs_number, readtime, url_keys, url_marks, score):
    
    sent_list = list(final_summary.split(sep='<hr>'))
    doc = Document()
    style = doc.styles['Normal']
    
    font = style.font
    font.name = 'Times New Roman'
    font.size = Pt(12)

    hd = doc.add_paragraph()
    hd.alignment = WD_ALIGN_PARAGRAPH.LEFT
    hd.add_run('Summary').bold = True

    if query != 'none':
        hd = doc.add_paragraph('Request: ' + "''" + query + "''")

    hd = doc.add_paragraph('Read time: ' + str(readtime) + ' min')
    hd = doc.add_paragraph('Information: ' + str(score) + '%')
    hd = doc.add_paragraph('Documents: ' + str(docs_number))
    
    hd = doc.add_paragraph('')
    
    hd.add_run('Keys:\n').underline = True
    
    for j in url_keys:
        if j != url_keys[-1]:
            add_hyperlink(hd, (str(j.split('/keyword/')[1]) + ', '), str(j.split('/keyword/')[0]), False)
        else:
            add_hyperlink(hd, (str(j.split('/keyword/')[1])), str(j.split('/keyword/')[0]), False)
        
    hd.add_run('\n\nBenchmarks:\n').underline = True
    
    for j in url_marks:
        if j != url_marks[-1]:
            add_hyperlink(hd, str(j.split('/keyword/')[1]) + ', ', str(j.split('/keyword/')[0]), False)
        else:
            add_hyperlink(hd, str(j.split('/keyword/')[1]), str(j.split('/keyword/')[0]), False)
    
    r = hd.add_run()
    for i in sent_list:
        hd.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

        if query != 'none':
            try:
                link = re.search(r"<a href=(.*?)target='_blank'", str(i)).group(1).replace(' ','')
                hd = doc.add_paragraph(striphtml(str(i)).replace('<hr>','').replace('<u>','').replace('More',''))               
                add_hyperlink(hd, 'More', link, True).add_run()
            except:
                link = ''
        if query == 'none':
            hd = doc.add_paragraph(striphtml(str(i)).replace('<hr>','').replace('<u>','').replace('More',''))    
         
    doc.save('docs/' + summary + '.docx')
    
    return True
#############/Doc preparation ##########

############## Get summary and tags ##########
def get_summary(rawtext, readtime):
    sentences = int(readtime/(np.median([len(i.split()) for i in nltk.sent_tokenize(rawtext)])/200))

    stemmer = Stemmer("english")
    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words("english")
    parser = PlaintextParser.from_string(' '.join(sent_tokenize(rawtext)[6:]), Tokenizer("english"))

    text_list = []
    for sentence in summarizer(parser.document, sentences):
        text_list.append(str(sentence))

    txt = ' '.join(sent_tokenize(rawtext)[:6]) + ' '+' '.join(text_list)

    z = 0
    output = []
    pdf_output = []

    for i in nltk.sent_tokenize(txt):
        z = z+1
        output.append('\n\n<hr>' + str(z) + '. ' + str(i))
        pdf_output.append('<hr>' + str(z) + '. ' + str(i))

    txt = ''.join(output) + '<hr>'
    pdf_txt = ''.join(pdf_output) + '<hr>'
    return txt, pdf_txt

def graph_keys(final_text, top_number):
    
    bigrams = list(nltk.ngrams(text_normalize(final_text.lower()),2))
    bigrams = [' '.join(i) for i in bigrams if (i[0]!=i[1])] 
    bigram_counts = collections.Counter(bigrams)
    
    df = pd.DataFrame(bigram_counts.most_common(len(bigram_counts)), columns=['bigram', 'count'])[:top_number]
    df['count'] = 100*df['count']/df['count'].sum().astype(int) 
    keys = ', '.join(list(df['bigram'].astype(str)))

    return keys

def get_entities(rawtext, tops):
    
    spacy_nlp = spacy.load('en_core_web_lg', disable=["tagger","parser"])
    nlp.max_length = 1000000000000
    doc = spacy_nlp(rawtext)

    ners = []
    for ent in doc.ents:
        if ent.label_ in ['ORG', 'PERSON']:
            ners.append(ent.text)
   
    ner_counts = collections.Counter(ners)

    df = pd.DataFrame(ner_counts.most_common(len(ner_counts)), columns=['ner_names', 'count'])[:tops]
    df['count'] = 100*df['count']/df['count'].sum().astype(int) 
    keys = ', '.join(list(df['ner_names'].astype(str)))
    
    return keys
##############/Get summary and tags ##########

############## Extend summary ##########
def get_ngrams(text): 
    grams = nltk.ngrams(text.split(), 2)
    grams_list = []
    for i in grams:
        grams_list.append(i)
    
    return grams_list 

def get_jaccard_sim(a,b):
    a, b = set(get_ngrams(a)), set(get_ngrams(b)) 
    c = a.intersection(b)

    return round(float(len(c)/len(a)), 2) 

def filter_text(page_txt, page_sum, threshold=0.5): 
    sent_list = []  
    for j in page_txt.split('.'):
        try:
            sim_score = get_jaccard_sim(j, page_sum)
        except:
            sim_score = 0
            
        if sim_score > threshold:
            sent_list.append(j + '.')
            
    return sent_list 
##############/Extend summary ##########

############## Add keyurls ################
def add_keyurls(final_keys, query):
    url_keys = []
    for i in final_keys.split(','):
        url = 'https://www.google.com/search?q=' + '+'.join(re.sub(r" ?\([^)]+\)", "", i).strip().split()) + '+' + query + '/keyword/' + i 
        url_keys.append(url)
        
    return url_keys     
##############/Add urls ################

############# Parse Patents #############
def parse_patents(query, keys_number, page_number, tag, cls=''):   
    
    # load driver
    driver = webdriver.Chrome(ChromeDriverManager().install())
    #/load driver

    # get urls 
    google_url = "https://www.google.com/search?q=" + query + "&num=" + str(page_number+1)
    driver.get(google_url)
    time.sleep(randint(1,5))

    soup = BeautifulSoup(driver.page_source,'lxml')
    result_div = soup.find_all('div', attrs={'class': 'g'})

    links = []
    titles = []
    errors = []

    descriptions = []
    for r in result_div:
        try:
            link = r.find('a', href=True)
            title = None
            title = r.find('h3')

            if isinstance(title,Tag):
                title = title.get_text()

            description = None
            description = r.find('span', attrs={'class': 'st'})

            if isinstance(description, Tag):
                description = description.get_text()

            if link != '' and title != '' and description != '':
                links.append(link['href'])
                titles.append(title)
                descriptions.append(description)

        except Exception as e:
            print(e)
            continue

    url_list = links[:(page_number)]
    #/ get urls

    # load content 
    txt = []
    titles = []
    for j in tqdm(url_list):
        delta = random.randint(1,1)
        time.sleep(delta)
        
        try:  
            if str(j).endswith('.pdf'): 
                file_data = parser.from_file(str(j))           
                t = file_data['content']
            else:
                t = parse_page(j,tag,cls)
            txt.append(t)
            titles.append(''.join(sent_tokenize(t)[:3]))
        except:
            print('Parsing error:',str(j))
            errors.append(str(j))
       
    googletext = re.sub('[^A-Za-z0-9.]+', ' ', '; '.join(txt))
    df = pd.DataFrame(list(zip(txt, url_list, titles)), columns=['text','link', 'page'])
    
    driver.stop_client()
    driver.close()
   
    return googletext, errors, df
#############/Parse Patents #############

Parameters:

In [3]:
UPLOAD_FOLDER = 'docs/'

readtime = 10

page_number = 10
keys_number = 10

process_time = 0

Feature toggles:

In [4]:
compress = True

wiki_sum = True
gogle_sum = True

arxiv_sum = True
patnt_sum = True

Create query:

In [5]:
query = input()

text summarization


Parse Wiki:

In [6]:
%%time

wikitext = ''
wikikeys = ''

df_wiki = pd.DataFrame()

if wiki_sum == True: 
    wiki_wiki = wikipediaapi.Wikipedia('en', extract_format=wikipediaapi.ExtractFormat.WIKI)

    red_query = "https://www.google.com/search?q=" + 'site:https://en.wikipedia.org ' + query + "&num=" + str(page_number+1)
    wiki_titles = parse_wiki(red_query)

    txts = []
    titles = []

    for i in tqdm(wiki_titles):  
        page_sum = wiki_wiki.page(i).summary
        page_txt = wiki_wiki.page(i).text
        sent_list = filter_text(page_txt, page_sum, threshold=0.5)
       
        titles.append(i)
        txts.append(''.join(sent_list).replace('\n', ''))        
    
    wikitext = ''.join(txts).replace('\n','') 

    if compress == True:
        wikitext = coref_res(filter_triplet(wikitext))

    wikikeys = graph_keys(wikitext, keys_number)
    wiki_entities = get_entities(wikitext, keys_number)

    url_list = [str('https://en.wikipedia.org/wiki/' + i)  for i in wiki_titles] 
    df_wiki = pd.DataFrame(list(zip(txts, url_list, titles)), columns=['text','link', 'page'])

    df_wiki.replace('', np.nan, inplace=True)
    df_wiki.dropna(inplace=True) 

    print('|Keywords|:', wikikeys, '\n')
    print('|Entities|:', wiki_entities, '\n')
    
winsound.Beep(2500, 1000)    

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\skamenshchikov\.wdm\drivers\chromedriver\win32\87.0.4280.88]
100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.55it/s]


|Keywords|: sentence extraction, multi document, document summarization, summary translation, extraction technique, summarization process, process set, set data, data create, create subset 

|Entities|: Automatic, ROUGE, Recall-Oriented Understudy 

Wall time: 41 s


Show Wiki:

In [7]:
df_wiki.head()

,text,link,page
0,Automatic summarization is the process of shor...,https://en.wikipedia.org/wiki/Automatic_summar...,Automatic_summarization
1,Multi-document summarization is an automatic p...,https://en.wikipedia.org/wiki/Multi-document_s...,Multi-document_summarization
2,"ROUGE, or Recall-Oriented Understudy for Gisti...",https://en.wikipedia.org/wiki/ROUGE_(metric),ROUGE_(metric)
3,Sentence extraction is a technique used for au...,https://en.wikipedia.org/wiki/Sentence_extraction,Sentence_extraction


Parse Arxiv:

In [14]:
%%time

arxivtext = ''
arxivkeys = ''

df_arxiv = pd.DataFrame()

if arxiv_sum == True:
    try:
        df_arxiv = parse_arxiv(query, page_number)[2]  
        arxivtext = ''.join(list(df_arxiv['text'])) 
    
        if compress == True:
            arxivtext = coref_res(filter_triplet(arxivtext))
    
        arxiv_entities = get_entities(arxivtext, keys_number)
        arxivkeys = graph_keys(arxivtext, keys_number)
    
        print('|Keywords|:', arxivkeys, '\n')
        print('|Entities|:', arxiv_entities, '\n')
    
    except:
        print('No data')
    
    df_arxiv.replace('', np.nan, inplace=True)
    df_arxiv.dropna(inplace=True)      
    
winsound.Beep(2500, 1000)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:34<00:00, 10.49s/it]


|Keywords|: relation extraction, text summarization, text generation, cx db, target language, model transfer, binary relation, simple binary, generation method, cross model 

|Entities|: CX_DB8, FACTGEN, Automatic, NIST, BERT, Email, making.kr, Transformer, CTRLsum, CNN 

Wall time: 1min 56s


Show Arxiv:

In [15]:
df_arxiv.head()

,text,link,page
0,edu Information retrieval (IR) for precision...,https://arxiv.org/abs/2012.09355,Literature Retrieval for Precision Medicine wi...
1,nl Automatic text summarization has enjoyed ...,https://arxiv.org/abs/2012.07619,What Makes a Good Summary? Reconsidering the F...
2,"In the healthcare domain, medical experts cr...",https://arxiv.org/abs/2012.07563,A Practical Approach towards Causality Mining ...
3,"kr Recently, sequence-to-sequence (seq2seq)...",https://arxiv.org/abs/2012.07280,Contrastive Learning with Adversarial Perturba...
4,Relation extraction is very important for kno...,https://arxiv.org/abs/2012.04821,Complex Relation Extraction: Challenges and Op...


Parse Google:

In [34]:
%%time

googletext = ''
googlekeys = ''

df_google = pd.DataFrame()

if gogle_sum == True:
    try:
        google_data = parse_google("https://www.google.com/search?q=", query, keys_number, page_number, 'body')
        df_google = google_data[2]
    
        googletext = ''.join(list(df_google['text']))
    
        if compress == True:
            googletext = coref_res(filter_triplet(googletext))
    
        google_entities = get_entities(googletext, keys_number)
        googlekeys = graph_keys(googletext, keys_number)
    
        print('|Keywords|:', googlekeys, '\n')
        print('|Entities|:', google_entities, '\n')
  
    
    except:
        print('No data')
    
df_google.replace('', np.nan, inplace=True)
df_google.dropna(inplace=True)      
    
winsound.Beep(2500, 1000)    

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\skamenshchikov\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache
  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Parsing error: https://www.imperial.ac.uk/pls/portallive/docs/1/18619759.PDF


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:48<00:00,  4.97s/it]


|Keywords|: text summarization, et al, speech summarization, topic model, topic word, document summarization, multi document, word sentence, tf idf, feature sentence 

|Entities|: Automatic, LSA, Pyramid, EM, Data, ASR

Acoustic Information, Greedy, Disember, Microsoft, ROUGE 

Wall time: 1min 27s


Show Google:

In [20]:
df_google.head(20)

,text,link,page
0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,https://www.imperial.ac.uk/pls/portallive/docs...,Trends and Applications of Automatic Text Summ...
1,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,https://core.ac.uk/download/pdf/11784914.pdf,403 Forbidden
2,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,https://core.ac.uk/download/pdf/81967205.pdf,403 Forbidden
3,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,https://www.fosteropenscience.eu/sites/default...,A Gentle Introduction to Text Summarization in...
4,Automatic Text Summarization (ATS) aims at gen...,https://www.cs.kent.ac.uk/people/staff/aaf/pub...,A Quick Introduction to Text Summarization in ...


In [23]:
str(df_google['text'][0])

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n105 \n\nSan \nAUTOMATIC TEXT SUMMARIZATION USING FEATURE-�\n\nBASED FUZZY EXTRACTION� 2.0", \n\nKey \nLadda Suanrnali\n\n1\n, Naomie Salim\n\n2\n, Mohammed Salem Binwahlan3 \n\nIvan \n\nrsity, IFaculty of Science and Technology \nSuan Dusit Rajabhat University \n\n295 Rajasrima Rd, Dusit, Bangkok, Thailand 10300 \n~ce", \n\n2.3Faculty of Computer Science and Infonnation System� \nUniversiti Teknologi Malaysia� \n\n81310 Skudai, Johor� \n\nE-mail: Iladda_sua@dusit.ac.th.2naomie@utm.my.3moham2007med@yahoo.com \n\nAbstract: Automatic text summarization is to compress the original text into a shorter version \n\nand help the user to quickly understand large volumes of infonnation. This paper focuses on the \n\nautomatic text summarization by sentence extraction with important features based on fuzzy \n\nlogic. In our experiment, we used 6 test documents in DUC2002 data set. Each document is \n\nprepared by p

Parse patents:

In [ ]:
%%time

patenttext = ''
patentkeys = ''

df_patent = pd.DataFrame()

# try:
#     z = parse_patents("site:https://patents.google.com " + query, keys_number, page_number, 'div', 'abstract')
#     patent_text = z[0] 
    
# #     patent_entities = get_entities(patent_text, keys_number)
# #     patentkeys = graph_keys(patent_text, keys_number)
    
# #     print('|Keywords|:', patentkeys, '\n')
# #     print('|Entities|:', patent_entities, '\n')
    
#     df_patent = z[2]
# excepthgjfhgjfhgjfhjfhjhs:
#     print('No data')
    
# df_patent.replace('', np.nan, inplace=True)
# df_patent.dropna(inplace=True)  

# winsound.Beep(2500, 1000)

Show Patents:

In [ ]:
df_patent.head()

Trend keys:

In [ ]:
%%time

a = (arxivtext + patenttext) 
b = (wikitext) 

specific_keys = get_entities(a, keys_number)
common_keys = get_entities(b, keys_number)

specific_keys = [re.sub(r" ?\([^)]+\)", "", i).strip() for i in specific_keys.split(',')]
common_keys = [re.sub(r" ?\([^)]+\)", "", i).strip() for i in common_keys.split(',')]

df1 = pd.DataFrame(list(zip(specific_keys)), columns =['specific'])
df2 = pd.DataFrame(list(zip(common_keys)), columns =['known'])

matches = fpd.fuzzy_merge(df1, df2, left_on=['specific'], right_on=['known'], ignore_case=True, keep='match', 
                         method='levenshtein', threshold=0.8, join='inner')

specific_reqs = list(set(specific_keys).difference(set(list(matches['specific']))))
specific_reqs = [i for i in specific_reqs if i[0].isupper()] + [i for i in specific_reqs if i[0].islower()]
specific_keys = ', '.join([i for i in specific_reqs if i])

print('|Trend keys|:', specific_keys, '\n')

winsound.Beep(2500, 1000)

Concatenate dataframes:

In [ ]:
df = df_wiki.append(df_google).append(df_arxiv).append(df_patent)

Get text and tags:

In [ ]:
%%time

final_text = (wikitext + arxivtext + googletext + patenttext)
final_entities = get_entities(final_text, keys_number)
final_keys = graph_keys(' '.join(text_normalize(final_text)), keys_number)

Define full reading time:

In [ ]:
full_readtime = len(nltk.sent_tokenize(final_text))*(np.median([len(i.split()) for i in nltk.sent_tokenize(final_text)])/200)

Get best reading time:

In [ ]:
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

score_list = []
read_list = []

for i in range(1, int(full_readtime), 10):
    
    report_summary = get_summary(final_text, i)[1]
    scores = scorer.score(report_summary, final_text)
    scores = round(100*list(list(scores.values())[0])[2])
    
    print('Readtime:', i, 'min')
    print('Rouge:', (str(scores) + '%'))
    print('\n')
    
    score_list.append(scores)
    read_list.append(round((i/full_readtime),2))
    
    if scores > sc_lim:
        break
        
print('|Recommended time|:', i, 'min') 
winsound.Beep(2500, 1000)

Plot calibration curve:

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(11,4))
axes[0].plot([i*full_readtime for i in read_list], score_list, '-o', markersize=9)
axes[1].plot(read_list, score_list, '-o', markersize=9)

axes[0].set_title('Rouge L, %')
axes[0].set_xlabel('Readtime, min')

axes[1].set_title('Rouge L,%')
axes[1].set_xlabel('Compression')

axes[0].grid()
axes[1].grid()

fig.tight_layout()

Enter readtime:

In [ ]:
readtime = input()

Get summary:

In [ ]:
report_summary = get_summary(final_text, int(readtime))[1]
scores = scorer.score(report_summary, final_text)
scores = round(100*list(list(scores.values())[0])[2])

print('Information extracted:', (str(scores) + ' %'))

winsound.Beep(2500, 1000)

Create keys with urls:

In [ ]:
url_keys = add_keyurls(final_keys, query)
mark_keys = add_keyurls(final_entities, query)  

ref_list = []
pdf_list = []

sent_list = list(report_summary.split(sep='<hr>'))[1:]

for i in sent_list[:-1]:
    try:
        df_score = df.copy()
        df_score['score'] = df_score['text'].apply(lambda x: css(i,x))
        df_score = df_score.sort_values(by=['score'], ascending=False)
        
        if str(df_score['link'].iloc[0]):
            pdf_list.append(str(i))
            ref_list.append(str(df_score['link'].iloc[0]))
    except:
        pdf_list.append('')

pdf_summary = ''.join(pdf_list)
winsound.Beep(2500, 1000)

Clean sentences from tags:

In [ ]:
text_list = [BeautifulSoup(i, "lxml").text.replace('More', '').strip().capitalize() for i in pdf_list]
text_list = [(i.replace(i.split()[0], '').strip().capitalize()) for i in text_list if i]

Create dataframe from tags and urls:

In [ ]:
df = pd.DataFrame(list(zip(ref_list, text_list)), columns=['link', 'text'])
df = df.sort_index(ascending=True).groupby('link', as_index=True).agg(lambda x: ' '.join(x))
df = df.reindex(list(unique_everseen(ref_list))).reset_index()

Add new sources:

In [ ]:
ref_list = []
pdf_list = []

trc = 0

for i in range(len(df)):
    trc = trc + 1
    pdf_list.append(str(trc) + '. ...' + str(str(df['text'].iloc[i]) + " <u><a href=" + str(df['link'].iloc[i]) + " target='_blank'>" + "More" + "</a></u>" + "<hr>"))
    ref_list.append(str(df['link'].iloc[i]))

pdf_summary = ''.join(pdf_list)

Save docx:

In [ ]:
save_doc(pdf_summary, 'summary', query, len(df), readtime, url_keys, mark_keys, scores)

Statistics:

In [ ]:
memory_size = int(len(final_text.encode('utf-8'))/1024)
docs_number = len(df)

print('Memory size:', memory_size, 'mb')
print('Number of documents:', docs_number)

winsound.Beep(2500, 5000)